## Discovery and Ordering Workflow

In this example workflow, a user will be shown how to:
* Search STAC items
* View pipelines
* Validate an order
* Check an order's estimated usage
* Place an order
* Cancel an order

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the discovery and ordering portions of the Maxar Geospatial Platform SDK

In [ ]:
mgp_discovery = interface.discovery_service
mgp_ordering = interface.order_service

#### Discover STAC items

The first thing we want to do is search the available STAC items via Discovery. Here we will set up some variables to make the discovery process more manageable

In [ ]:
bbox = "-105,40,-104,41"
datetime = "2015-01-01T00:00:00Z/2016-01-01T00:00:00Z"
collections = "wv02"
where = "eo:cloud_cover<20"
limit = 10

We will utilize the stac_search function and iterate through a list of all of the ids that match our criteria and set a variable for the first id in that list

In [ ]:
features = mgp_discovery.stac_search(bbox=bbox, datetime=datetime, where=where, collections=collections, orderby='id', limit=limit)
features_list = []
for feature in features['features'][:limit]:
    features_list.append(feature['id'])
wv02_feature = features_list[0]
print(wv02_feature)

Now that we have a single feature, we can narrow down the metadata for just that feature with the get_stac_item funciton. We want to know what the supported pipelines are for this feature, so we will narrow the result down to show the pipelines and their names

In [ ]:
desired_feature = mgp_discovery.get_stac_item(collection_id=collections, item_id=wv02_feature)
for i in desired_feature['links']:
    print(i['rel'])
    print(i['href'])

For this workflow, we want to use the *map-ready* pipeline. We now know that the feature can be ordered via the map-ready pipeline

If you are unfamiliar with a pipeline and what is needed for the order to work, you can use the get_pipeline_details function to determine what information needs to be passed in. The namespace of the pipeline can be found in the *href* link from the above function. It is the section of the link between *pipelines* and the pipeline name, in this case, with a namespace of *imagery* and a name of *map-ready*. With our pipeline identified, we can narrow down the response of the get_pipeline_details function to show us the necessary information needed to place an order

In [ ]:
pipeline = mgp_ordering.get_pipeline_details(namespace='imagery', name='map-ready')
required_info = pipeline['data']['settings_schema']['required']
print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = pipeline['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

From this response we can see that the inventory_ids and the customer_description are required for order placement, and that inventory_ids is an array of strings while customer_description is a string

#### Ordering a feature

Before we fully place an order, we want to verify that the order is formatted properly and will be accepted, and what the estimated cost of the order will be. First we will set up some variables to make the requests more manageable. The output_config variable will be a dictionary where we store our S3 information, the notificaitons variable will be a list of a dictionary that will store our email information, and the metadata variable will be a dictionary that stores our project_id, or order name

In [ ]:
namespace = 'imagery'
name = 'map-ready'
output_config = {"output_config": {"amazon_s3": {"bucket": "yourS3BucketName", "prefix": "your/S3/bucket/prefix/name"}}}
settings = {
            "settings": {
                "inventory_ids": [wv02_feature],
                "customer_description": "your order description"
            }
        }
notifications = [{"type": "email", "target": "your.email@address.com", "level": "FINAL_ONLY"}]
metadata = {"metadata": {"project_id": "your order name"}}

We will now validate the order before placing it by setting the validate argument to *True*

In [ ]:
order_validate = mgp_ordering.place_order(namespace=namespace, name=name, output_config=output_config, settings=settings, 
                                         notifications=notifications, metadata=metadata, validate=True)
print(order_validate)

The status of the order is shown as RECEIVED, indicating that the order passes the validation, meaning we can place the order. One last thing to check before we place the order is to check the estimated usage of the order

In [ ]:
order_estimate = mgp_ordering.get_usage_estimate(namespace=namespace, name=name, output_config=output_config, settings=settings, 
                                         notifications=notifications, metadata=metadata)
print(order_estimate)

From this response we can see that the order will use 1922.748 sqkm of usage. We can now place the order

In [ ]:
order = mgp_ordering.place_order(namespace=namespace, name=name, output_config=output_config, settings=settings, 
                                         notifications=notifications, metadata=metadata)
order_id = order['data']['id']
print(order)

After the order has been placed, we may want to check the full details of the submitted order. This can be done with the get_order_details function and passing in the order id generated from the placement of the order

In [ ]:
check_order = mgp_ordering.get_order_details(order_id=order_id)
print(check_order)

We also may want to occasionally check up on the status of the order. We can do this by utilizing the get_order_events function and checking the message of the response

In [ ]:
check_order_events = mgp_ordering.get_order_events(order_id=order_id)
print(check_order_events)

In some instances, we may want to cancel an order. Some pipelines unfortunately do not allow cancelling of orders. To check, we can run the get_all_pipelines function and see if cancelling is allowed for our pipeline

In [ ]:
all_pipelines = mgp_ordering.get_all_pipelines()
for pipeline in all_pipelines['data']['pipelines']:
    if pipeline['name'] == name:
        if pipeline['orders_cancellable']:
            print("Pipeline {} allows cancellations".format(name))
        else:
            print("Pipeline {} does not allow cancellations".format(name))

Assuming the pipeline allows for cancellations, we can run the cancel_order function to cancel the desired order

In [ ]:
cancel_order = mgp_ordering.cancel_order(order_id)
print(cancel_order)